In [1]:
IMG_SIZE = (224, 224, 3)

In [2]:
from keras import applications as kapps
from skimage.io import imread
mn_model = kapps.MobileNetV2(input_shape = IMG_SIZE, include_top=False)
from ipywebrtc import ImageRecorder, CameraStream

Using TensorFlow backend.


ImportError: cannot import name 'Video'

In [ ]:
if True:
    camera = CameraStream.facing_user(audio=False)
else:
    camera = CameraStream.facing_environment(audio=False)
image_recorder = ImageRecorder(stream=camera)
image_recorder

In [ ]:
import ipywidgets as ipw

In [ ]:
ipw.__version__